## Import Packages

In [7]:
import pandas as pd
import numpy as np
import plotly.express as px # quick plotting of data 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression  # basic classification model for initial exploration
from sklearn.ensemble import RandomForestClassifier # classification model for higher performance and  generability
from sklearn.model_selection import GridSearchCV # for parameter tuning 
from sklearn.model_selection import cross_val_score # cross validation of predictive models 
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from datetime import datetime 
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook

## Import Data 

In [171]:
df_train = pd.read_csv("Data Sets/Blight/train.csv", encoding = 'latin1', low_memory = False)  #standard ucf-8 encoding cannot decode csv
df_test = pd.read_csv("Data Sets/Blight/test.csv")
df_latlons = pd.read_csv("Data Sets/Blight/latlons.csv")
df_addresses = pd.read_csv("Data Sets/Blight/addresses.csv")

## Feature Exploration

Features avaliable for classification:

<br>
       'ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail'
<br>

Of these features, features related to the address can be dropped after the table is joinned with the latlon table as information regarding the physical location of the address is presented in the latlon table as lat & lon values.

Additionally, fields associated with the payment of the fines can also be dropped those are fields associated with the payment itself being made and won't be avaliable come testing.

Lastly, the field associated with the violator him or herself should also be dropped to avoid labelling the possibility of complaince on who the violator is. 

Rows with no data on complaince dropped as NA on complaince means that violator is given incorrect verdict.

In [131]:
df_train.isna().sum() # check for columns with null vals 

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
payment_amount                     0
b

### Investigating 'agency_name' on Compliance

In [5]:
agency_cols = ['agency_name', 'compliance']
df_agency = df_train[agency_cols]
df_agency = df_agency.dropna()
dum = pd.get_dummies(df_agency)
dum.corr().loc['compliance']  ## 'agency_name' has little to no effect on compliance

compliance                                                    1.000000
agency_name_Buildings, Safety Engineering & Env Department   -0.055637
agency_name_Department of Public Works                        0.046939
agency_name_Detroit Police Department                         0.038672
agency_name_Health Department                                -0.005559
agency_name_Neighborhood City Halls                          -0.000699
Name: compliance, dtype: float64

### Investigating 'inspector_name' on Compliance 

In [236]:
inspector_cols = ['inspector_name', 'compliance']
df_inspector = df_train[inspector_cols]
df_inspector = df_inspector.dropna()
dum = pd.get_dummies(df_inspector)
(dum.corr().loc['compliance'] > 0.05).sum() ## 'inspector_name' has little to no effect on compliance

1

### Investigating 'violation_code' on Compliance 

In [235]:
violation_cols = ['violation_code', 'compliance']
df_violation = df_train[violation_cols]
df_violation = df_violation.dropna()
dum = pd.get_dummies(df_violation)
(dum.corr().loc['compliance'] > 0.05).sum() ## 'violation_code' has little to no effect on compliance -- thus 
                                           ##  'violation description' should also have no effect on compliance

1

### Investigating 'grafitti_status' on Compliance

In [41]:
pd.value_counts(df_train.grafitti_status)  ## number of samples with valid information for this feature too small 

GRAFFITI TICKET    1
Name: grafitti_status, dtype: int64

### Investigating 'disposition' on Compliance

In [140]:
disposition_cols = ['disposition', 'compliance']
df_disposition = df_train[~df_train.compliance.isna()]
df_disposition = df_disposition[disposition_cols]
dum = pd.get_dummies(df_disposition)
dum.corr().loc['compliance'] ## 'disposition' has an effect on compliance


compliance                                        1.000000
disposition_Responsible (Fine Waived) by Deter    0.124956
disposition_Responsible by Admission              0.238997
disposition_Responsible by Default               -0.335455
disposition_Responsible by Determination          0.202819
Name: compliance, dtype: float64

### Investigating datetime Feature Fields 

#### Effect of Specific Day of the Week on Compliance

In [217]:
cols_dates = ['ticket_issued_date','hearing_date', 'compliance']
df_temp = df_train.loc[:,cols_dates]
df_temp.loc[:,cols_dates[0]] = pd.to_datetime(df_temp[cols_dates[0]])
df_temp.loc[:,cols_dates[1]] = pd.to_datetime(df_temp[cols_dates[1]]) # Set up table for investigation

df_temp.loc[:, cols_dates[0]] = df_temp[cols_dates[0]].dt.weekday
df_temp.loc[:, cols_dates[1]] = df_temp[cols_dates[1]].dt.weekday
(df_temp[['ticket_issued_date' , 'compliance']].corr(), df_temp[['hearing_date' , 'compliance']].corr())
## Day of the week has little to no effect on compliance 

(                    ticket_issued_date  compliance
 ticket_issued_date            1.000000    0.006522
 compliance                    0.006522    1.000000,
               hearing_date  compliance
 hearing_date      1.000000   -0.025301
 compliance       -0.025301    1.000000)

#### Effect of Specific Month of the Year on Compliance

In [220]:
cols_dates = ['ticket_issued_date','hearing_date', 'compliance']
df_temp = df_train.loc[:,cols_dates]
df_temp.loc[:,cols_dates[0]] = pd.to_datetime(df_temp[cols_dates[0]])
df_temp.loc[:,cols_dates[1]] = pd.to_datetime(df_temp[cols_dates[1]]) # Set up table for investigation

df_temp.loc[:, cols_dates[0]] = df_temp[cols_dates[0]].dt.month
df_temp.loc[:, cols_dates[1]] = df_temp[cols_dates[1]].dt.month
(df_temp[['ticket_issued_date' , 'compliance']].corr(), df_temp[['hearing_date' , 'compliance']].corr())
## Specific month has little to no effect on compliance 

(                    ticket_issued_date  compliance
 ticket_issued_date            1.000000   -0.040555
 compliance                   -0.040555    1.000000,
               hearing_date  compliance
 hearing_date      1.000000   -0.032841
 compliance       -0.032841    1.000000)

#### Effect of the Difference Between Issued Date and Hearing Date on Compliance

In [230]:
cols_dates = ['ticket_issued_date','hearing_date', 'compliance']
df_temp = df_train.loc[:,cols_dates]
df_temp.loc[:,cols_dates[0]] = pd.to_datetime(df_temp[cols_dates[0]])
df_temp.loc[:,cols_dates[1]] = pd.to_datetime(df_temp[cols_dates[1]]) # Set up table for investigation

df_temp.loc[:,'ticket_issued_date'] = pd.DatetimeIndex(df_temp['ticket_issued_date']).astype(int) ## Covert time to UNIX
df_temp.loc[:,'hearing_date'] = pd.DatetimeIndex(df_temp['hearing_date']).astype(int) 
df_temp.loc[:,'issued_to_hearing'] = df_temp[cols_dates[1]] - df_temp[cols_dates[0]]

scaler = MinMaxScaler()
df_temp.loc[:,'issued_to_hearing'] = scaler.fit_transform(df_temp[['issued_to_hearing']])

df_temp[['compliance', 'issued_to_hearing']].corr()


,compliance,issued_to_hearing
compliance,1.000000,0.095732
issued_to_hearing,0.095732,1.000000


## Model Fitting and Testing - Logistic Regression

In [173]:
## Joining addresses to latlon to get latlon of each address
df_addresses = df_addresses.merge(df_latlons, how = 'inner', left_on = 'address', right_on = 'address').dropna()


## Joining addresses to df_train & df_test to that they will contain latlon 
df_train = df_train.merge(df_addresses, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id' )
df_test = df_test.merge(df_addresses, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id')


## Removing records where case is dropped 
df_train = df_train[~(df_train['compliance'].isna())]


## Test cols with addresses and other qualitative fields removed along with fields with na
cols_train = [ 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
'clean_up_cost', 'disposition','judgment_amount', 'lat', 'lon']

cols_all = cols_train + ['compliance']

df_train_X = pd.get_dummies(df_train[cols_train])

df_train_y = df_train['compliance']

logreg = LogisticRegression(solver = 'lbfgs', max_iter = 300, penalty = 'l2')

cross_val_score(logreg, df_train_X, df_train_y,
                cv = ShuffleSplit(n_splits = 5, test_size = 0.2),
               error_score = 'raise') ## custom cv to ensure real randomization of splits

array([0.93301226, 0.93363773, 0.93332499, 0.93401301, 0.93360645])

## Compare perfomance to Dummy Classifier

In [65]:
dummy_clas = DummyClassifier(strategy = 'most_frequent')
dummy_clas.fit(X_train, y_train)
cross_val_score(dummy_clas, df_train_X, df_train_y, cv= 5) 

array([0.92744785, 0.92744558, 0.92747459, 0.92747459, 0.92747459])

The logistic regression model can consistently outperform the dummy classifier but only by a small amount.

## Performance on Positive Class only Training Set - Logistic Regression

In [155]:
X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_y)
custom_test = pd.concat([X_test, y_test], axis = 1)

custom_test = custom_test[custom_test['compliance'] == 1]
custom_test_X = custom_test.iloc[:,:-1]
custom_test_y = custom_test['compliance']


logreg_temp = LogisticRegression(solver = 'lbfgs', max_iter = 300, penalty = 'l2')

logreg_temp.fit(X_train, y_train)
logreg_temp.score(custom_test_X, custom_test_y) 



0.10425531914893617

The model is able to predict ~11% of positive class -- almost perfect classifier of negative class.

## Model Fitting and Testing - Random Forest Model

In [161]:
for_clas = RandomForestClassifier(n_jobs = -1, n_estimators = 10) ## n_job = -1 to utlize all cores, n_estimator set 
                                                                  ## to default to silence update warning

cross_val_score(for_clas, df_train_X, df_train_y,
                cv = ShuffleSplit(n_splits = 5, test_size = 0.2),
                error_score = 'raise') ## custom cv to ensure real randomization of splits




array([0.93789092, 0.94036152, 0.9388604 , 0.94079935, 0.93914186])

## Hyperparameter Tuning - Random Forest Classifier

In [58]:
parameters = {'n_estimators': [50, 100, 200], 'max_depth' : [12, 24, 36]}
for_clas = RandomForestClassifier(n_jobs = -1)
clf = GridSearchCV(for_clas, parameters)
clf_result = clf.fit(X_train, y_train)
best_params = clf_result.best_params_

C:\Users\g36e3.000\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.



In [59]:
best_params

{'max_depth': 24, 'n_estimators': 200}

In [181]:
parameters = {'min_samples_split': [2, 5, 9], 'min_samples_leaf' : [3, 7, 13]}
for_clas = RandomForestClassifier(n_jobs = -1, n_estimators = 200, max_depth = 24)
clf = GridSearchCV(for_clas, parameters)
clf_result = clf.fit(X_train, y_train)
best_params = clf_result.best_params_

C:\Users\g36e3.000\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.



In [182]:
best_params

{'min_samples_leaf': 3, 'min_samples_split': 9}

max_depth of 24 allows for the highest score by optimzing the proportation of correct positive class guesses and negative class guesses. n_estimators of 200 allows for high accuracy without sacrificing run-time. 

## Tuned Model Performance - Random Forest Classifier

In [183]:
for_clas = RandomForestClassifier(n_jobs = -1, max_depth = 24, n_estimators = 200,
                                 min_samples_leaf = 3, min_samples_split = 9) ## n_job = -1 to utlize all cores 

cross_val_score(for_clas, df_train_X, df_train_y,
                cv = ShuffleSplit(n_splits = 5, test_size = 0.2),
                error_score = 'raise') ## custom cv to ensure real randomization of splits




array([0.94336377, 0.94567801, 0.94739805, 0.94520891, 0.94480235])

Tuned model is objectively better at classifying.

## Performance on Positive Class only Training Set - Random Forest Classifier

In [170]:
custom_test = pd.concat([X_test, y_test], axis = 1)

custom_test = custom_test[custom_test['compliance'] == 0]
custom_test_X = custom_test.iloc[:,:-1]
custom_test_y = custom_test['compliance']

for_clas_temp = RandomForestClassifier(n_estimators = 200,
                                       max_depth  = 24, n_jobs = -1)

for_clas_temp.fit(X_train, y_train)
for_clas_temp.score(custom_test_X, custom_test_y) 


0.9932166890982503

The tuned RFC is approximately 3 times better at classifing the positive class than the LOGREG classifier and is able to identify ~35% of the positive class -- it's an almost perfect classifier of the negative class.

## Use Tuned Model for Real Test Data Prediction

In [167]:
for_clas = RandomForestClassifier(n_estimators = 200, max_depth = 24, n_jobs = -1)
for_clas.fit(df_train_X, df_train_y)

df_test_X = pd.get_dummies(df_test[cols_train])

predict_result = for_clas.predict(df_test_X.drop(
    columns = ['disposition_Responsible (Fine Waived) by Admis',
       'disposition_Responsible - Compl/Adj by Default',
       'disposition_Responsible - Compl/Adj by Determi',
               'disposition_Responsible by Dismissal'
              ]))

pd.DataFrame(index = df_test['ticket_id'], data = {'compliance' : predict_result})


,compliance
ticket_id,
284932,0.0
285362,0.0
285361,0.0
285338,0.0
285346,0.0
...,...
376496,0.0
376497,0.0
376499,0.0
